# classificationbox
https://machineboxio.com/docs/classificationbox

classificationbox is [online learning](https://en.wikipedia.org/wiki/Online_machine_learning) (supervised learning), it works also with little data but it build the classifier function so it needs more data. for my bird monirtoring project my 2 classes would be bird/no_bird, and I just post the image data

Classifiers can be made to help solve a wide range of example use cases, for example:
* Learn about how your company is perceived by grouping tweets into positive and negative
* Automatically group photos of cats and dogs
* Group emails into spam and non-spam categories
* Build a classifier to detect the language of a piece of text based on previously taught examples

```
sudo docker pull machinebox/classificationbox

sudo docker run -p 8080:8080 -e "MB_KEY=$MB_KEY" machinebox/classificationbox
```
http://localhost:8080/

In [1]:
import requests

In [2]:
IP = 'localhost'
#IP = '192.168.0.30'
PORT = '8080'
CLASSIFIER = 'classificationbox'
MODEL_ID = "sentiment1"

MODEL_CREATION_URL = 'http://{}:{}/{}/models'.format(IP, PORT, CLASSIFIER)
MODEL_TEACH_URL = 'http://{}:{}/{}/models/{}/teach'.format(IP, PORT, CLASSIFIER, MODEL_ID)
MODEL_PREDICT_URL = 'http://{}:{}/{}/models/{}/predict'.format(IP, PORT, CLASSIFIER, MODEL_ID)
MODEL_STATS_URL = 'http://{}:{}/{}/models/{}/stats'.format(IP, PORT, CLASSIFIER, MODEL_ID)

# Model creation
First we create the model, which simply consists of providing some basic config and a list of classes

In [3]:
model = {"id": MODEL_ID,
         "name": "sentimentModel",
         "options": {
             "ngrams": 1,
             "skipgrams": 1},
         "classes": [
             "class1",
             "class2",
             "class3"]}

In [4]:
model

{'id': 'sentiment1',
 'name': 'sentimentModel',
 'options': {'ngrams': 1, 'skipgrams': 1},
 'classes': ['class1', 'class2', 'class3']}

In [5]:
requests.post(MODEL_CREATION_URL, json=model).json() # Create the model

{'success': True,
 'id': 'sentiment1',
 'name': 'sentimentModel',
 'options': {'ngrams': 1, 'skipgrams': 1},
 'predict_only': False}

Check the model was created by making a get

In [6]:
requests.get(MODEL_CREATION_URL).json()

{'success': True,
 'models': [{'id': '5b0ce5d8023d4e35', 'name': '5b0ce5d8023d4e35'},
  {'id': 'sentiment1', 'name': 'sentimentModel'}]}

# Teaching the model
Post some data to the model, possible inputs (features) are:
* number - An integer or floating point number (e.g. likes, age, size)
* text - A blob of text that will broken down into tokens (e.g. body of an article)
* keyword - A string value that will not be tokenized (e.g. cities, authors)
* list - A comma separated list of keywords (e.g. visit history articleA,articleB,articleC,...)
* image_url - The URL to an image (which will be downloaded by Classificationbox)
* image_base64 - A string containing the base64 encoded image data

In [7]:
data_to_teach = {
    "class": "class1",
    "inputs": [
        {"key": "user_age", "type": "number", "value": "25"},
        {"key": "user_interests", "type": "list", "value": "music,cooking,ml"},
        {"key": "user_location", "type": "keyword", "value": "London"}
    ]}

In [8]:
data_to_teach

{'class': 'class1',
 'inputs': [{'key': 'user_age', 'type': 'number', 'value': '25'},
  {'key': 'user_interests', 'type': 'list', 'value': 'music,cooking,ml'},
  {'key': 'user_location', 'type': 'keyword', 'value': 'London'}]}

In [9]:
requests.post(MODEL_TEACH_URL, json=data_to_teach).json() # Create the model

{'success': True}

# Model prediction
Now make a prediction 

In [10]:
data_query = {
    "limit": 10,
    "inputs": [
        {"key": "user_age", "type": "number", "value": "25"},
        {"key": "user_interests", "type": "list", "value": "music,cooking,ml"},
        {"key": "user_location", "type": "keyword", "value": "London"}]}

requests.post(MODEL_PREDICT_URL, json=data_query).json()

{'success': True,
 'classes': [{'id': 'class1', 'score': 0.42109},
  {'id': 'class3', 'score': 0.293062},
  {'id': 'class2', 'score': 0.285848}]}

OK it predicted `class1`

Lets check the model stats

In [11]:
requests.get(MODEL_STATS_URL).json()

{'success': True,
 'predictions': 1,
 'examples': 1,
 'classes': [{'name': 'class1', 'examples': 1}]}

# Download the model
You can download the model state into a binary file for backup, and upload previously saved models.

In [12]:
MODEL_STATE_URL = 'http://{}:{}/classificationbox/state/{}'.format(IP, PORT, MODEL_ID)

In [13]:
response = requests.get(MODEL_STATE_URL)
filename = "model_{}.classificationbox".format(MODEL_ID)
print(filename)
open(filename, 'wb').write(response.content)

model_sentiment1.classificationbox


1211

In [14]:
ls

Classification box introduction.ipynb  cat.1.jpg
Classificationbox images.ipynb         cat.2.jpg
Facebox.ipynb                          model_sentiment1.classificationbox
Facebox_HA_development.ipynb           monkey.jpg
Tagbox tag images.ipynb                thebeatles.jpg
